### Creating and Persisting an ML Model

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

Create a subset of features as an example.

In [2]:
# include = ['health','absences','age','G3']
# df.drop(columns=df.columns.difference(include), inplace=True)  # only using 3 features

In [3]:
# df.info

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [7]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
df.drop(columns=['G1', 'G2', 'G3'], inplace=True)

Drop the G3 score

Import scikit-learn and build a random forest classifer

In [8]:
# encode string labels
from sklearn import preprocessing
for column in df:
    if df[column].dtype == object:
        le = preprocessing.LabelEncoder()
        le.fit(df[column])
        df[column] = le.transform(df[column])
df.describe()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,...,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,0.116456,0.473418,16.696203,0.777215,0.288608,0.896203,2.749367,2.521519,2.169620,2.281013,...,0.832911,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,0.184810
std,0.321177,0.499926,1.276043,0.416643,0.453690,0.305384,1.094735,1.088201,1.227473,0.863542,...,0.373528,0.472300,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,0.388636
min,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,16.000000,1.000000,0.000000,1.000000,2.000000,2.000000,2.000000,2.000000,...,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,0.000000
50%,0.000000,0.000000,17.000000,1.000000,0.000000,1.000000,3.000000,2.000000,2.000000,2.000000,...,1.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000
75%,0.000000,1.000000,18.000000,1.000000,1.000000,1.000000,4.000000,3.000000,3.000000,3.000000,...,1.000000,1.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,0.000000
max,1.000000,1.000000,22.000000,1.000000,1.000000,1.000000,4.000000,4.000000,4.000000,4.000000,...,1.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,1.000000


In [9]:
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_selection import RFECV

dependent_variable = 'qual_student'
# separate to train and test set
from sklearn.model_selection import train_test_split
x = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.1)
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
params = {
    #'n_estimators': [100, 200, 500, 800, 1000],
    #'max_depth' : [4,5,6,7,8],
    'max_features': [i for i in range(1, 31)]
}
clf = rf()
rfecv = RFECV(clf, step=1, cv=k_fold, scoring='f1')
rfecv.fit(x_train, y_train)

include = np.array([i for i in range(30)])
include = include [rfecv.support_]
include = np.array(df.columns)[include]
print(include)



['sex' 'Fedu' 'reason' 'schoolsup' 'famsup' 'activities']


/home/fustinose/workspace/17313/team/fall-2021-hw4-arbitrary-name/env/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  warnings.warn(


In [10]:
dependent_variable = 'qual_student'
# include = ['health','absences','age']
# include = ['sex','Fedu', 'reason', 'schoolsup', 'famsup', 'activities']
print(include)
include = np.append(include, dependent_variable)
print(include)
print(include.shape)

['sex' 'Fedu' 'reason' 'schoolsup' 'famsup' 'activities']
['sex' 'Fedu' 'reason' 'schoolsup' 'famsup' 'activities' 'qual_student']
(7,)


In [11]:
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import GridSearchCV, train_test_split
df.drop(columns=df.columns.difference(include), inplace=True)
x = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = rf(n_estimators=1000)
param_grid = {
                 'n_estimators': [100, 250, 600, 800, 1000],
                 'max_depth': [3, 5, 7, 9]
             }
grid = GridSearchCV(clf, params = {
    ''
}, scoring='f1', cv=k_fold)
clf.fit(x_train, y_train)
# print(cv)


RandomForestClassifier(n_estimators=1000)

In [12]:
import sklearn
pred = clf.predict(x_test)
sklearn.metrics.f1_score(y_test, pred, average='binary')

0.0

In [ ]:

pred = grid.predict(x_test)
sklearn.metrics.f1_score(y_test, pred, average='binary')

NameError: name 'grid' is not defined

It's not very good! We didn't even cross validate. You'll need to do better :)
Let's export this model so we can use it in a microservice (flask api)

In [ ]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, 'dockerfile/apps/model.pkl')

['dockerfile/apps/model.pkl']

In [ ]:
query_df = pd.DataFrame({ 'age' : pd.Series(1) ,'health' : pd.Series(15) ,'absences' : pd.Series(10)})

In [ ]:
pred = clf.predict(query_df)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.